# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.232750251930                   -0.50    8.0         
  2   -7.249988966282       -1.76       -1.40    1.0   8.63ms
  3   -7.251209676201       -2.91       -1.97    2.0   10.2ms
  4   -7.251226616326       -4.77       -2.07    2.0   10.5ms
  5   -7.251320895743       -4.03       -2.54    1.0   8.55ms
  6   -7.251335936678       -4.82       -2.91    1.0   8.51ms
  7   -7.251338728735       -5.55       -3.76    1.0   8.52ms
  8   -7.251338793072       -7.19       -4.29    2.0   10.0ms
  9   -7.251338798249       -8.29       -4.77    2.0   11.3ms
 10   -7.251338798604       -9.45       -5.08    2.0   10.4ms
 11   -7.251338798694      -10.05       -5.75    2.0   12.7ms
 12   -7.251338798701      -11.17       -5.76    3.0   17.0ms
 13   -7.251338798704      -11.53       -6.19    1.0   8.90ms
 14   -7.251338798704      -12.33       -6.82    2.0   9.93ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.050446858231724505
[ Info: Arnoldi iteration step 2: normres = 0.7454137686327454
[ Info: Arnoldi iteration step 3: normres = 0.6887182870988178
[ Info: Arnoldi iteration step 4: normres = 0.3307965263790484
[ Info: Arnoldi iteration step 5: normres = 0.24809350036404532
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (1.36e-02, 4.82e-02, 1.35e-01, 1.89e-01, 7.06e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3864332593918393
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.95e-03, 2.30e-01, 3.02e-01, 4.91e-02, 1.73e-02)
[ Info: Arnoldi iteration step 7: normres = 0.09547321947249239
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (2.38e-04, 2.66e-02, 1.09e-02, 1.73e-02, 8.24e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11811067848471765
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.22e